In [1]:
import os
import pandas as pd
import torch

os.chdir("C:\\Users\\ndgig\\Repositories\\phrase_foundry")
print(os.getcwd())

%load_ext autoreload
%autoreload 2

from phrase_foundry import PhraseFoundry

C:\Users\ndgig\Repositories\phrase_foundry


In [2]:
splits = {
    "train": "train_consumer_complaints.csv",
    "test": "test_consumer_complaints.csv",
}
df = pd.read_csv(
    "hf://datasets/Johnade/consumer_complaints_cfpb/" + splits["test"]
).sample(2000, random_state=32)

docs = df["consumer_complaint_narrative"].to_list()

docs[:5]

['Navient is my student loan provider. I also owe from a student loan from a  XXXX  college, the loan borrower is Nelnet. The Nelnet lo an is from  XXXX , this is what I owed then {$3200.00}. I was paying on it for a long time then I started going back to school in  XXXX  to get a  XXXX   XXXX  so I deferred. I still owe {$4000.0 0} on the Nelnet loan, I asked Nelent to  forgive my loan and they denied also. Since I have had  XXXX , they have emailed me about deferring every time I get behind so I defer. What was a  XXXX  loan is now close t o  XXXX . This is because of deferring payments and interest. Today I am behind three months. The beginning of the year I asked for a  XXXX  forbearance for the school year and was denied. Today I looked into def  erring and if I defer again the balance of my loan will be close t  o 70,000. How am I ever to get this paid in my lifetime when the interest is outrageous?',
 'In accordance with the Fair Credit Reporting Act. The Account below violated 

In [52]:
if "model" in locals():
    del model
model = PhraseFoundry(
    # "sentence-transformers/all-mpnet-base-v2",
    "sentence-transformers/paraphrase-MiniLM-L3-v2",
    device="cuda",
    invalid_start_token_pattern=r"^@",
)
model

In [53]:
model.tokenizer.convert_tokens_to_ids("theology")

8006

In [54]:
model.invalid_start_token_ids

array([1030])

In [55]:
model.tokenizer.convert_ids_to_tokens(model.invalid_start_token_ids)

['@']

In [4]:
# inputs, token_embeds, attns = ng.encode(docs, batch_size=512, amp=False)
# token_embeds.shape, attns.shape

In [5]:
# attns

In [6]:
# i = 0
# tokens = [ng.tokenizer.decode(x) for x in inputs["input_ids"][i]]
# token_attns = attns[i]
# pd.Series(token_attns, index=tokens).sort_values(ascending=False).head(25)

In [7]:
samp_idx, ngrams, ngram_vecs = model.encode_extract(
    docs,
    batch_size=512,
    ngram_range=(4, 6),
    amp=True,
    amp_dtype=torch.bfloat16,
)


ngram_vecs

Encoding:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting:   0%|          | 0/3 [00:00<?, ?it/s]

array([[ 0.10303904,  0.10579471, -0.04340914, ...,  0.08290204,
        -0.11373062, -0.08914515],
       [ 0.12873742,  0.05143999, -0.10486422, ...,  0.07682122,
        -0.10404699, -0.04945968],
       [ 0.13308784,  0.13264012, -0.11765344, ...,  0.09166077,
        -0.11608183, -0.08161601],
       ...,
       [ 0.19086039,  0.23453529,  0.02137025, ...,  0.08104255,
        -0.09196191,  0.04007579],
       [ 0.1581165 ,  0.2838741 ,  0.01274911, ...,  0.07281285,
        -0.12851848,  0.01004857],
       [ 0.14604613,  0.24925499, -0.0014254 , ...,  0.07588337,
        -0.1328897 ,  0.02108472]], dtype=float32)

In [8]:
samp_idx, ngrams, ngram_vecs = model.encode_extract(
    docs, batch_size=512, ngram_range=(4, 6), amp=True, amp_dtype=torch.bfloat16
)


ngram_vecs

Encoding:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting:   0%|          | 0/3 [00:00<?, ?it/s]

array([[ 0.10303904,  0.10579471, -0.04340914, ...,  0.08290204,
        -0.11373062, -0.08914515],
       [ 0.12873742,  0.05143999, -0.10486422, ...,  0.07682122,
        -0.10404699, -0.04945968],
       [ 0.13308784,  0.13264012, -0.11765344, ...,  0.09166077,
        -0.11608183, -0.08161601],
       ...,
       [ 0.19086039,  0.23453529,  0.02137025, ...,  0.08104255,
        -0.09196191,  0.04007579],
       [ 0.1581165 ,  0.2838741 ,  0.01274911, ...,  0.07281285,
        -0.12851848,  0.01004857],
       [ 0.14604613,  0.24925499, -0.0014254 , ...,  0.07588337,
        -0.1328897 ,  0.02108472]], dtype=float32)

In [9]:
ngrams[samp_idx == 2]

array(['i have been making', 'have been making payments',
       'been making payments on', 'making payments on my',
       'payments on my student', 'on my student loan',
       'my student loan and', 'student loan and it', 'loan and it has',
       'and it has been', 'it has been reported', 'has been reported i',
       "been reported i '", "reported i'm", "i'm 3", "' m 3 months",
       'm 3 months behind', '3 months behind i', 'months behind i haven',
       "behind i haven '", "i haven't", "haven't made", "' t made mid",
       't made mid payments', 'made mid payments at',
       'mid payments at all', 'payments at all for', 'at all for the',
       'all for the last', 'for the last 4', 'the last 4yr', 'last 4yrs',
       '4yrs now', 'now they want to', 'they want to find',
       'want to find me', 'to find me in', 'find me in default',
       'i have been making payments', 'have been making payments on',
       'been making payments on my', 'making payments on my student',
    

In [10]:
print(docs[0])

Navient is my student loan provider. I also owe from a student loan from a  XXXX  college, the loan borrower is Nelnet. The Nelnet lo an is from  XXXX , this is what I owed then {$3200.00}. I was paying on it for a long time then I started going back to school in  XXXX  to get a  XXXX   XXXX  so I deferred. I still owe {$4000.0 0} on the Nelnet loan, I asked Nelent to  forgive my loan and they denied also. Since I have had  XXXX , they have emailed me about deferring every time I get behind so I defer. What was a  XXXX  loan is now close t o  XXXX . This is because of deferring payments and interest. Today I am behind three months. The beginning of the year I asked for a  XXXX  forbearance for the school year and was denied. Today I looked into def  erring and if I defer again the balance of my loan will be close t  o 70,000. How am I ever to get this paid in my lifetime when the interest is outrageous?


In [11]:
ngrams[samp_idx == 0], ngram_vecs[samp_idx == 0]

(array(['navient is', 'is my student loan', 'my student loan provider',
        'student loan provider.', 'loan provider. i', 'provider. i also',
        '. i also owe', 'i also owe from', 'also owe from a',
        'owe from a student', 'from a student loan', 'a student loan from',
        'student loan from a', 'loan from a xx', 'from a xxxx',
        'a xxxx college', 'xxxx college,', 'college, the loan',
        ', the loan borrow', 'the loan borrower', 'loan borrower is',
        'borrower is ne', 'is nelnet', 'nelnet.', '. the neln',
        'the nelnet', 'nelnet lo', 'lo an is from', 'an is from xx',
        'is from xxxx', 'from xxxx,', 'xxxx, this', ', this is what',
        'this is what i', 'is what i owed', 'what i owed then',
        'i owed then {', 'owed then { $', 'then { $ 320', '{ $ 3200',
        '$ 3200.', '3200. 00', '. 00 }.', '00 }. i', '}. i was',
        '. i was paying', 'i was paying on', 'was paying on it',
        'paying on it for', 'on it for a', 'it for 

Encoding:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting:   0%|          | 0/2 [00:00<?, ?it/s]

array([[ 0.03413197, -0.06059905, -0.04677777, ...,  0.01644105,
        -0.04462195,  0.01331554],
       [ 0.09459738,  0.14775304,  0.019829  , ...,  0.10801277,
        -0.05441972, -0.01667406]], dtype=float32)

In [18]:
from sklearn.neighbors import NearestNeighbors


nn = NearestNeighbors(radius=0.45, metric="cosine")
nn.fit(ngram_vecs)
queries = model.encode_queries(
    ["interest rate is too high"],
    amp=True,
    amp_dtype=torch.bfloat16,
)
dists, idx = nn.radius_neighbors(queries)
ngrams[idx[0]], samp_idx[idx[0]]

Encoding:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting:   0%|          | 0/1 [00:00<?, ?it/s]

(array(['when the interest is', 'the interest is outrageous',
        'interest is outrageous?', '00 } in interest', '} in interesttha',
        'would earn interest rates', 'earn interest rates that',
        'interest rates that are', 'charge more interest which',
        'to reduce interest rates', 'reduce interest rates could',
        'interest rates could just', 'and the interest rate',
        'the interest rate was', 'interest rate was supposed',
        '% interest for over', 'want my interest rate',
        'my interest rate reduced', 'interest rate reduced.',
        'raised my interest rate', 'my interest rate nearly',
        'interest rate nearly xx', 'rate nearly xxxx',
        'rise in interest rates', 'in interest rates that',
        'interest rates that started', 'loan and the interest',
        'and the interest rate', 'the interest rate is',
        'interest rate is high', 'rate is high because',
        'pay high cost interest', 'high cost interest rates',
      